In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import ops
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet


target_shape = (224, 224)

ImportError: cannot import name 'ops' from 'keras' (c:\Users\Tymof\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\__init__.py)

In [4]:
!pip install tensorflow --upgrade

^C
